# OECD Scrapping

## Import all necessary libraries

In [1]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from bs4 import SoupStrainer
import re
import csv
import numpy as np
import pandas as pd
import time
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Manually adding the 11 URLs of the 11 differents topics




In [2]:
my_url = ['https://www.oecdbetterlifeindex.org/topics/housing',
          'https://www.oecdbetterlifeindex.org/topics/education/',
          'https://www.oecdbetterlifeindex.org/topics/life-satisfaction/',
          'https://www.oecdbetterlifeindex.org/topics/income/',
          'https://www.oecdbetterlifeindex.org/topics/environment/',
          'https://www.oecdbetterlifeindex.org/topics/safety/',
          'https://www.oecdbetterlifeindex.org/topics/jobs/',
          'https://www.oecdbetterlifeindex.org/topics/civic-engagement/',
          'https://www.oecdbetterlifeindex.org/topics/work-life-balance/',
          'https://www.oecdbetterlifeindex.org/topics/community/',
          'https://www.oecdbetterlifeindex.org/topics/health/']

## Scrapping of the 11 topics pages

In [3]:
topicID = 1
AllTopicsID = []
AllTopics = []
AllBackgroundText = []
AllBackgroundLinks = []
AllFMOLinks = []


for url in my_url:
    uClient = uReq(url)
    page_HTML = uClient.read()
    uClient.close()
    page_soup_html = soup(page_HTML, "html.parser")
    
    ###Get the Topics ID
    
    AllTopicsID.append(topicID)
    topicID = topicID + 1
    
    ###Get the Topics
    
    All_h1_text = page_soup_html.find_all("h1")
    for x in All_h1_text:
        title = x.text
    AllTopics.append(title)
    
    ###Get the Topics' background text
    
    All_topic_text = page_soup_html.findAll("div",{"class":"span8"})
    
    backToString = ''
    backToString = All_topic_text[0].text
    
    foundBackgroundText = 0
        
    backgroundText = ''
        
    Backgroundpattern = re.compile(r'(?=[a-zA-Z]).+?(?=For more information on estimates)', flags = re.DOTALL)
    Backgroundmatches = Backgroundpattern.finditer(backToString)

    for Backgroundmatch in Backgroundmatches:
        backgroundText = Backgroundmatch.group(0)
        foundBackgroundText = 1
            
    if foundBackgroundText == 0:
        backgroundText = backToString
    
    AllBackgroundText.append(backgroundText)
 
    ###Get the Topics' background links

    backToString = ''
    backToString = str(All_topic_text[0])
    
    foundBackgroundLink = 0  
    allLinksOfThePage = ''
        
    Linkspattern = re.compile(r'(?<=href=\").*?(?=<\/a>)')
    Linksmatches = Linkspattern.finditer(backToString)

    for Linksmatch in Linksmatches:
        allLinksOfThePage = allLinksOfThePage + Linksmatch.group(0).replace('">', '  ') + ' | '
        foundBackgroundLink = 1
    
    if foundBackgroundLink == 0:
        allLinksOfThePage = 'No link found in Background'
        
    AllBackgroundLinks.append(allLinksOfThePage)
    
    ###Get the Find Out More Links and Titles
    
    All_find_out_more_text = page_soup_html.findAll("div",{"class":"span3 offset1"})

    backToString = ''
    backToString = str(All_find_out_more_text[0])
    
    allLinksOfTheFOM = ''
        
    FOMLinkspattern = re.compile(r'(?<=href=\").*?(?=<\/a>)')
    FMOLinksmatches = FOMLinkspattern.finditer(backToString)

    everyOtherLink = 1

    for FMOLinksmatch in FMOLinksmatches:
        if (everyOtherLink % 2) == 1:        
            allLinksOfTheFOM = allLinksOfTheFOM + FMOLinksmatch.group(0).replace('">', '  ') + ' | '
        everyOtherLink = everyOtherLink + 1
        
    AllFMOLinks.append(allLinksOfTheFOM)

## Scrapping of Indicator with Selenium script

In [4]:
chrome_path = "/Users/jeremybernard/Documents/Eurostat/Driver/chromedriver"
options = Options()

options.add_argument('--disable-gpu')
options.add_argument('--log-level=3')
options.add_argument("--user-data-dir=C:/Users/lea/AppData/Local/Google/Chrome/User Data")
options.add_argument("profile-directory=Profile 1")
options.add_argument("--disable-extensions")


AllIndicators = []

for START_URL in my_url:
    driver = webdriver.Chrome(chrome_path, options = options)

    driver.get(START_URL)

    time.sleep(5)

    soup_level = soup(driver.page_source, 'html')

    page_content = soup_level.find_all('a', {'class': 'info'})


    indicatorlist = []
    indicator = ''

    for x in page_content[1:]:
        text = x.text
        if len(indicatorlist) == 0:
            indicatorlist.append(text)
        else: 
            if text in indicatorlist and len(indicatorlist)>1:
                break
            indicatorlist.append(text)

        
    for i in indicatorlist:
        indicator = indicator + str(i) + ' | '

    AllIndicators.append(indicator)

    driver.quit()
    
    time.sleep(5)

<ipython-input-4-4eb41a55385d>:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path, options = options)


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


## Add the data of Topics into an csv file

In [ ]:
OECDTopics = pd.DataFrame(list(zip(AllTopicsID, AllTopics, AllBackgroundText, AllBackgroundLinks, AllIndicators, AllFMOLinks)), #allNewsText)),
                                                      columns=['Topic ID', 'Name of Topic', 'Background', 'BG + FMI Links', 'Indicators', 'FOM LInks']) #, 'News Content']) #, 'News Content', 'News inside links'])
 
#print(OECDTopics)

    
table = OECDTopics.to_csv('OECDTopics.csv', index=False)

## Scrapping data of each countries

In [ ]:
AllTopics = []
AllCountriesInTopic = []
AllKeyFindings = []
# AllBetterPol = []
# AllMoreRess = []
AllLinks = []

for url in my_url:
    uClient = uReq(url)
    page_HTML = uClient.read()
    uClient.close()
    page_soup_html = soup(page_HTML, "html.parser")
    
    All_h1_text = page_soup_html.find_all("h1")
    for x in All_h1_text:
        title = x.text

    All_h2_text = page_soup_html.find_all("h2")

    countries = ''
    for x in All_h2_text:
        countries = countries + str(x)

    Countrypattern = re.compile(r'(?<= [–|-] ).+?(?= \<a)', flags = re.DOTALL)
    Countrymatches = Countrypattern.finditer(countries)

    AllCountries = []
    
    for Countrymatch in Countrymatches:
        Country = Countrymatch.group(0)
        AllCountries.append(Country)
        AllTopics.append(title)
        AllCountriesInTopic.append(Country)
    
    All_topic_text = page_soup_html.findAll("div",{"class":"span8"})

    for x in range(1, len(AllCountries)+1, 1):
        keyFindings = All_topic_text[x].text
        
        keyFindingsWithBalise = str(All_topic_text[x])
    
        foundLink = 0  
        allLinksOfThePage = ''
        
        Linkspattern = re.compile(r'(?<=href=\").*?(?=<\/a>)')
        Linksmatches = Linkspattern.finditer(keyFindingsWithBalise)
    
        for Linksmatch in Linksmatches:
            allLinksOfThePage = allLinksOfThePage + Linksmatch.group(0).replace('"><i class="icon-document"></i>', ' ') + ' | '
            foundLink = 1
    
        if foundLink == 0:
            allLinksOfThePage = 'No links'
                        
        AllKeyFindings.append(keyFindings)
        AllBetterPol.append(BetterPolText)
        AllMoreRess.append(MoreRessText)
        AllLinks.append(allLinksOfThePage)

## Add the data of each country to an csv file

In [ ]:
OECDTopicsByCountry = pd.DataFrame(list(zip(AllTopics, AllCountriesInTopic, AllKeyFindings, AllLinks)), #allNewsText)),
                                                      columns=['Topic', 'Country', 'Key Findings', 'All Links']) #, 'News Content']) #, 'News Content', 'News inside links'])
 
#print(OECDTopicsByCountry)

    
table = OECDTopicsByCountry.to_csv('OECDTopicsByCountry.csv', index=False)